### Imports

In [11]:
#%load_ext autoreload
#%autoreload 2
#%pip install numpy
#%pip install matplotlib
#%pip install scipy
import numpy as np
import json
import sys
sys.path.append('/content/drive/MyDrive/Thesis')
from NeuroStitch import NeuronStitcher
from matplotlib import pyplot as plt

In [12]:
import json

# Percorso del file JSON del neurone
neuronPath = 'ms1821_alignment_soloVPM_nocontours_neuritescorrected.json'

# Carica il file JSON del neurone
with open(neuronPath) as fp:
    neuron = json.load(fp)

### Settings

In [13]:
# the neuron MUST BE in json format, use morphology viewer to convert ASC/XML/DAT to json.
neuronPath = 'ms1821_alignment_soloVPM_nocontours_neuritescorrected.json'

# where to save the stitched neuron to, again only json format is supported, use morphology viewer to convert to XML.
stitchedNeuronPath = '/content/drive/MyDrive/Thesis/stitches/stitched.json'

# somaLocations are the approximate (x,y,z)-coordinates of one or more somata.
# For each somaLocation the algorithm will find the nearest 'ending' to start the stitching process from.
# Use somaLocations = None for an automated guess based on soma contour lines of a single soma.
somaLocations = None
#somaLocations = [
#    [1.2659e4,-2086.2,2407.9], # axon near soma contour #1
#    [1.2669e4,-1719.7,2492.9], # axon 6280 near soma contour #2
#    [1.2595e4,-1562.6,2638.2], # axon 6307 near soma contour #6
#    [1.26e4,-1882.7,2539.4], # axon near soma contour #4
#    [1.2758e4,-1778.3,2647.7], # axon near soma contour #5
#    [1.296e4,-1470,2700.2] # isolated piece of axon
#]

# manualStitches list pairs of coordinates that definitely need to be stitched.
# The stitcher will lookup the pair of endings that are nearest to the pair of coordinates.
manualStitches = [
#    [[1.3415e4,-1523.2,2047],[1.3433e4,-1535.6,2058.6]], # lines 4203-4410
#    [[1.2951e4,-1887.1,2544.7],[1.2936e4,-1878.5,2554.8]], # lines 6285-6304
#    [[1.3754e4,-1841.2,2251],[1.3787e4,-1833.3,2235.2]] # lines 5526-5262
]

# if larger than 1, this allows the neuron to fork at a stitch location.
maxStitchesPerEnding = 1

# section thickness, only used in the penalty settings below
sectionThickness = 50

# Depending on the usecase, zero, one and/or more stitches per ending should be allowed.
# A penalty can be applied to ensure that multiple stitches don't happen too often.
stitchesPerEndingPenalty = {
    1:0,
    2:sectionThickness/2
}

# The usual section jump for a stitch is 1 (i.e. stitch pieces in adjacent sections), so no penalty for that.
# Within-section stitches have a jump of 0, if undesirable they can be penalized
# Larger jumps can be allowed but no need to penalize them since they are already spatially separated
# Example:
#   sectionJumpPenalty = { 0:sectionThickness/2, 1:0, 2:0 }
sectionJumpPenalty = {
    0:sectionThickness/2,
    1:0,
    2:0
}

# Exception: do not apply sectionJump penalty if endings are closer than noPenaltyThreshold
sectionJumpNoPenaltyThreshold = 1

## Options related to the actual stitching process
# whether to actually connect the pieces of neurite together
connectPieces=False
# only create stitch if the distance is smaller than maxConnectDistance (-1 ignores this)
maxConnectDistance=100
# create a marker at all putative stitch locations
markAsMarker=False,
# create a piece of neurite to indicate stitch locations, the integer value indicates the type of neurite to create
# where 4 (apical dendrite) is a common choice.
markAsNeurite=4,
# mark stitched pieces of neurite in color
markAsColor=False

### Load the neuron

In [14]:
neuronPath = 'ms1821_alignment_soloVPM_nocontours_neuritescorrected.json'
with open(neuronPath) as fp:
  neuron = json.load(fp)

neuronStitcher = NeuronStitcher(neuron,['um','RAS','corner'])

### Find best stitch locations

In [15]:
neuronStitcher.verifyAll()

All parent indices are valid.
All reverse properties are valid.


In [16]:
# La variabile [2] indica che si sta considerando solo il neurite di tipo assone,
# utilizzare 3 per i dendriti e 4 per i dendriti apicali.
axonPieces = neuronStitcher.getPiecesOfNeurite([2])

# Chiamata semplificata senza soglie angolari multiple
final_result = neuronStitcher.findBestStitches(
    axonPieces,
    somaLocations=somaLocations,
    stitchesPerEndingPenalty=stitchesPerEndingPenalty,
    manualStitchLocations=manualStitches,
    sectionJumpPenalty=sectionJumpPenalty,
    noPenaltyThreshold=sectionJumpNoPenaltyThreshold,
    angleWeight=0.7,
    proportionTolerance=2.0,
    initial_weight_ratio=0.3
)

if final_result is not None:
    # Stampa i risultati
    print("Results:")
    print(f"  Number of stitches: {len(final_result['stitches'])}")
    print(f"  Number of unconnected endings: {final_result['numUnconnectedEndings']}")
else:
    print("No results found.")


Attempting connections
Eps stimato: 13.406591523177298
Numero di cluster identificati: 294
Numero di outliers: 124
Density threshold stimato: 6.0
Densità media calcolata: 9.66326530612245
Zona densa rilevata con densità media: 9.66326530612245
Eps stimato: 13.406591523177298
Numero di cluster identificati: 294
Numero di outliers: 124
Density threshold stimato: 6.0
Densità media calcolata: 9.66326530612245
Zona densa rilevata con densità media: 9.66326530612245
Nessuna connessione trovata per 1.
Nessuna connessione trovata per 2.
Nessuna connessione trovata per 5.
[DEBUG] Processing sorted attractor point for pieceId 6: 5, 2
Nessuna connessione trovata per 7.
Nessuna connessione trovata per 8.
[DEBUG] Processing sorted attractor point for pieceId 9: 551, 2
[DEBUG] Processing sorted attractor point for pieceId 10: 944, 2
[DEBUG] Processing sorted attractor point for pieceId 11: 946, 5
[DEBUG] Processing sorted attractor point for pieceId 6: 5, 5
[DEBUG] Processing sorted attractor point 

[DEBUG] Processing sorted attractor point for pieceId 70: 873, 4[DEBUG] Processing sorted attractor point for pieceId 77: 133, 1
PieceId: 69, Best connection found: 69-0 to 902-5 with score 0.9231127398123427
Connect 69-0 to 902-5 at growth angle 82.28; and direction angle 82.46 at attractor distance 9.38. 462 to go
Nessuna connessione trovata per 84.
[DEBUG] Processing sorted attractor point for pieceId 85: 738, 4
[DEBUG] Processing sorted attractor point for pieceId 77: 871, 2
[DEBUG] Processing sorted attractor point for pieceId 85: 45, 1
[DEBUG] Processing sorted attractor point for pieceId 77: 871, 5

[DEBUG] Processing sorted attractor point for pieceId 77: 70, 2
[DEBUG] Processing sorted attractor point for pieceId 70: 872, 2
[DEBUG] Processing sorted attractor point for pieceId 85: 45, 4
[DEBUG] Processing sorted attractor point for pieceId 77: 70, 5
[DEBUG] Processing sorted attractor point for pieceId 77: 133, 0
[DEBUG] Processing sorted attractor point for pieceId 59: 54, 1


[DEBUG] Processing sorted attractor point for pieceId 192: 255, 2[DEBUG] Processing sorted attractor point for pieceId 191: 210, 5
[DEBUG] Processing sorted attractor point for pieceId 134: 509, 1

[DEBUG] Processing sorted attractor point for pieceId 168: 588, 0
[DEBUG] Processing sorted attractor point for pieceId 191: 860, 0
[DEBUG] Processing sorted attractor point for pieceId 168: 96, 2
PieceId: 134, Best connection found: 134-0 to 847-4 with score 0.9147130236262551
Connect 134-0 to 847-4 at growth angle 75.30; and direction angle 76.90 at attractor distance 6.26. 430 to go
[DEBUG] Processing sorted attractor point for pieceId 197: 252, 4
[DEBUG] Processing sorted attractor point for pieceId 197: 711, 3
[DEBUG] Processing sorted attractor point for pieceId 191: 210, 1
[DEBUG] Processing sorted attractor point for pieceId 192: 710, 3
[DEBUG] Processing sorted attractor point for pieceId 168: 913, 4
[DEBUG] Processing sorted attractor point for pieceId 197: 252, 3
[DEBUG] Processin

[DEBUG] Processing sorted attractor point for pieceId 303: 283, 4[DEBUG] Processing sorted attractor point for pieceId 328: 327, 3
[DEBUG] Processing sorted attractor point for pieceId 330: 353, 3
[DEBUG] Processing sorted attractor point for pieceId 328: 330, 0
[DEBUG] Processing sorted attractor point for pieceId 330: 333, 4
[DEBUG] Processing sorted attractor point for pieceId 334: 358, 1

[DEBUG] Processing sorted attractor point for pieceId 330: 329, 4
[DEBUG] Processing sorted attractor point for pieceId 330: 353, 2
[DEBUG] Processing sorted attractor point for pieceId 334: 329, 0
[DEBUG] Processing sorted attractor point for pieceId 330: 333, 3
[DEBUG] Processing sorted attractor point for pieceId 328: 352, 1
[DEBUG] Processing sorted attractor point for pieceId 330: 353, 5
[DEBUG] Processing sorted attractor point for pieceId 303: 283, 1
[DEBUG] Processing sorted attractor point for pieceId 334: 355, 5
[DEBUG] Processing sorted attractor point for pieceId 328: 327, 5
[DEBUG] Pr

[DEBUG] Processing sorted attractor point for pieceId 473: 31, 2[DEBUG] Processing sorted attractor point for pieceId 489: 34, 3

[DEBUG] Processing sorted attractor point for pieceId 473: 31, 5
[DEBUG] Processing sorted attractor point for pieceId 489: 34, 2
[DEBUG] Processing sorted attractor point for pieceId 473: 29, 2
[DEBUG] Processing sorted attractor point for pieceId 467: 77, 4
[DEBUG] Processing sorted attractor point for pieceId 462: 133, 2
[DEBUG] Processing sorted attractor point for pieceId 473: 29, 5
[DEBUG] Processing sorted attractor point for pieceId 462: 133, 5
PieceId: 462, Best connection found: 462-0 to 684-0 with score 0.7988360655585287
Connect 462-0 to 684-0 at growth angle 2.76; and direction angle 73.96 at attractor distance 9.68. 343 to go
Nessuna connessione trovata per 490.
[DEBUG] Processing sorted attractor point for pieceId 491: 227, 1
[DEBUG] Processing sorted attractor point for pieceId 467: 77, 1
[DEBUG] Processing sorted attractor point for pieceId 

[DEBUG] Processing sorted attractor point for pieceId 638: 154, 4[DEBUG] Processing sorted attractor point for pieceId 646: 643, 3

[DEBUG] Processing sorted attractor point for pieceId 638: 547, 2
[DEBUG] Processing sorted attractor point for pieceId 646: 639, 4
[DEBUG] Processing sorted attractor point for pieceId 638: 541, 1
[DEBUG] Processing sorted attractor point for pieceId 640: 108, 1
[DEBUG] Processing sorted attractor point for pieceId 655: 657, 4
[DEBUG] Processing sorted attractor point for pieceId 638: 547, 5
[DEBUG] Processing sorted attractor point for pieceId 655: 505, 4
[DEBUG] Processing sorted attractor point for pieceId 646: 548, 0
[DEBUG] Processing sorted attractor point for pieceId 646: 647, 1
[DEBUG] Processing sorted attractor point for pieceId 640: 100, 0
[DEBUG] Processing sorted attractor point for pieceId 640: 643, 2
[DEBUG] Processing sorted attractor point for pieceId 638: 547, 4
[DEBUG] Processing sorted attractor point for pieceId 655: 657, 3
[DEBUG] Pr

[DEBUG] Processing sorted attractor point for pieceId 749: 750, 2[DEBUG] Processing sorted attractor point for pieceId 771: 787, 0
[DEBUG] Processing sorted attractor point for pieceId 736: 799, 2
[DEBUG] Processing sorted attractor point for pieceId 736: 100, 0
[DEBUG] Processing sorted attractor point for pieceId 736: 640, 5
[DEBUG] Processing sorted attractor point for pieceId 771: 229, 1

[DEBUG] Processing sorted attractor point for pieceId 771: 783, 3
[DEBUG] Processing sorted attractor point for pieceId 772: 783, 4
[DEBUG] Processing sorted attractor point for pieceId 736: 941, 3
[DEBUG] Processing sorted attractor point for pieceId 772: 851, 1
[DEBUG] Processing sorted attractor point for pieceId 749: 807, 0
[DEBUG] Processing sorted attractor point for pieceId 736: 799, 1
[DEBUG] Processing sorted attractor point for pieceId 736: 217, 0
[DEBUG] Processing sorted attractor point for pieceId 771: 239, 4
[DEBUG] Processing sorted attractor point for pieceId 749: 224, 0
[DEBUG] Pr

[DEBUG] Processing sorted attractor point for pieceId 878: 879, 2[DEBUG] Processing sorted attractor point for pieceId 908: 762, 2
[DEBUG] Processing sorted attractor point for pieceId 899: 553, 2

[DEBUG] Processing sorted attractor point for pieceId 884: 877, 4
[DEBUG] Processing sorted attractor point for pieceId 884: 881, 0
[DEBUG] Processing sorted attractor point for pieceId 884: 880, 0
[DEBUG] Processing sorted attractor point for pieceId 899: 875, 0
[DEBUG] Processing sorted attractor point for pieceId 878: 879, 3
[DEBUG] Processing sorted attractor point for pieceId 908: 762, 3
[DEBUG] Processing sorted attractor point for pieceId 878: 879, 4
[DEBUG] Processing sorted attractor point for pieceId 884: 877, 3
[DEBUG] Processing sorted attractor point for pieceId 884: 892, 0
[DEBUG] Processing sorted attractor point for pieceId 899: 553, 3
[DEBUG] Processing sorted attractor point for pieceId 908: 762, 4
[DEBUG] Processing sorted attractor point for pieceId 878: 879, 5
[DEBUG] Pr

###                     SAVE THE NEURON

In [19]:
stitchedNeuronPath10 = 'stitched_direction_minore_growth_mejor.json'

In [20]:
# Verifica che ci siano risultati
if final_result is not None and isinstance(final_result, dict):
    stitches = final_result['stitches']

    # Applica le suture ai pezzi di assone
    neuronStitcher.applyStitches(
        axonPieces,
        stitches,
        connectPieces=connectPieces,
        maxConnectDistance=maxConnectDistance,
        markAsMarker=markAsMarker,
        markAsNeurite=markAsNeurite,
        markAsColor=markAsColor
    )

    # Salva il neurone modificato nel file JSON
    with open(stitchedNeuronPath10, 'wt') as fp:
        json.dump(neuronStitcher.asDict(), fp)

    print(f"Neurone modificato salvato in {stitchedNeuronPath10}")
else:
    print("Non sono stati trovati risultati validi.")


ERROR, piece 946 already has an incoming connection.
ERROR, piece 551 already has an incoming connection.
ERROR, piece 60 already has an incoming connection.
ERROR, piece 262 already has an incoming connection.
ERROR, piece 329 already has an incoming connection.
ERROR, piece 329 already has an incoming connection.
ERROR, piece 353 already has an incoming connection.
ERROR, piece 355 already has an incoming connection.
ERROR, piece 77 already has an incoming connection.
ERROR, piece 680 already has an incoming connection.
ERROR, piece 529 already has an incoming connection.
ERROR, piece 639 already has an incoming connection.
ERROR, piece 555 already has an incoming connection.
ERROR, piece 58 already has an incoming connection.
ERROR, piece 59 already has an incoming connection.
ERROR, piece 210 already has an incoming connection.
ERROR, piece 690 already has an incoming connection.
ERROR, piece 179 already has an incoming connection.
ERROR, piece 807 already has an incoming connectio